In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, time, json
import pandas as pd
from openai import OpenAI
from tqdm.auto import tqdm
import spacy
import pandas as pd

import sys
sys.path.append("../../")
import os
from src.functional import free_gpu_cache
from src.utils import env_utils

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
story_template = "<actor> works in a busy restaurant. A customer asks <actor> for <obj_belief>. <actor> grabs an opaque <container> and fills it with <obj_belief>. A coworker, who didn't hear the customer's request, swaps the <obj_belief> in the <container> with <obj_true> while <actor> is attending to another task. <actor> does not see the coworker swapping the contents in the <container> and also can't see what is in it without opening it."

container_options = [
    "pitcher", "bottle", "flask", # "vial", "beaker", "jug", "cup", "mug", "glass", "tumbler", "thermos", "carafe", "decanter", "growler", "canteen", "gourd", "goblet", "stein", "chalice", "tankard", "shot glass", "snifter",
]
milk_options = [
    "oat milk", "almond milk", "soy milk", "rice milk", "whole milk", 
]
juice_options = [
    "orange juice", "mango juice", "apple juice", "grape juice", "cranberry juice", "pomegranate juice", "guava juice", "passion fruit juice", "lychee juice", "kiwi juice", "watermelon juice", "cantaloupe juice", "honeydew juice", "pineapple juice", "coconut juice", "papaya juice", "pear juice", "peach juice", "plum juice", "apricot juice", "cherry juice", "blueberry juice", "blackberry juice", "raspberry juice", "strawberry juice"
]
alcohol_options = [
    "beer", "wine", "whiskey", "vodka", "rum", "tequila", "gin", "brandy", "cognac", "liqueur", "absinthe", "sake", "soju", "baijiu", "shochu", "akvavit", "aquavit", "arak", "arrack", "bitters", "bourbon", "calvados", "cachaça", "grappa", "mezcal", "pisco", "rakia", "slivovitz", "ouzo", "raki", "schnapps", "scotch", "vermouth", "amaretto", "baileys", "chartreuse", "cointreau", "curacao", "frangelico", "grand marnier", "kahlua", "sambuca", "triple sec", "campari", "jagermeister", "malibu", "midori", "ouzo", "pimms", "southern comfort", "tuaca", "umbrella drink", "zima", "zombie", "zinfandel", "zinger"
]
obj_options = [
    set(milk_options), set(juice_options), set(alcohol_options)
]

In [4]:
from src.models import load_LM
import torch

model_key = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_key = "meta-llama/Meta-Llama-3-70B-Instruct"

lm = load_LM(
    model_key=model_key,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.31s/it]

loaded /home/local_arnab/Codes/00_MODEL/meta-llama/Meta-Llama-3-8B-Instruct | size: 5332.516 MB


In [19]:
with open(os.path.join(env_utils.DEFAULT_DATA_DIR, "english_names.json"), "r") as f:
    names = json.load(f)

len(names)

103

In [20]:
from src.functional import prepare_input
single_token_names = []

for name in names:
    tok = prepare_input(prompts=f" {name}", tokenizer=lm)
    if tok.input_ids.shape[1] == 2 and tok.input_ids[0][0] == lm.tokenizer.bos_token_id:
        single_token_names.append(name)

f"filtered {len(names)} names to {len(single_token_names)} single token names"

'filtered 103 names to 103 single token names'

In [21]:
with open(os.path.join(env_utils.DEFAULT_DATA_DIR, "english_names.json"), "w") as f:
    json.dump(list(single_token_names), f)

with open(os.path.join(env_utils.DEFAULT_DATA_DIR, "english_names.json"), "r") as f:
    names = json.load(f)

In [25]:
from src.dataset import SampleV2
import random
import itertools

obj_choose_2 = {
    "milk": list(itertools.combinations(milk_options, 2)),
    "juice": list(itertools.combinations(juice_options, 2)),
    "alcohol": list(itertools.combinations(alcohol_options, 2)),
}

unique_samples = []

for actor in names:
    for container in container_options:
        for obj_type in obj_choose_2:
            obj_pairs = random.sample(obj_choose_2[obj_type], 6)
            for obj_belief, obj_true in obj_pairs:
                story = story_template.replace("<actor>", actor)
                story = story.replace("<container>", container)
                story = story.replace("<obj_belief>", obj_belief)
                story = story.replace("<obj_true>", obj_true)
        
                sample = SampleV2(
                    story=story,
                    actor=actor,
                    container=container,
                    obj_belief=obj_belief,
                    obj_true=obj_true,
                )

                unique_samples.append(sample)
    #         break
    #     break
    # break

unique_samples = list(set(unique_samples))

len(unique_samples)

5561

In [26]:
len(unique_samples)

5561

In [27]:
from src.dataset import DatasetV2

dataset = DatasetV2(samples = unique_samples)

In [31]:
# question, answer = dataset[2]
question, answer = dataset.__getitem__(2, set_ans="yes")
print(question)
print(answer)

Instruction: Keep track of people's knowledge defined in the story. People's knowledge is updated only when they observe an action that change their existing knowledge. To answer the question following the story, choose "yes" or "no" after the "Answer:" tag.

Story: Frank works in a busy restaurant. A customer asks Frank for rice milk. Frank grabs an opaque flask and fills it with rice milk. A coworker, who didn't hear the customer's request, swaps the rice milk in the flask with whole milk while Frank is attending to another task. Frank does not see the coworker swapping the contents in the flask and also can't see what is in it without opening it.

Question: Does Frank believe that there is rice milk in the flask?
Answer:
yes


In [29]:
from src.utils import env_utils

with open(os.path.join(env_utils.DEFAULT_DATA_DIR, "dataset_v2.json"), "w") as f:
    json.dump(dataset.to_dict(), f)

In [30]:
with open (os.path.join(env_utils.DEFAULT_DATA_DIR, "dataset_v2.json"), "r") as f:
    dataset_dict = json.load(f)

loaded_dataset = DatasetV2.from_dict(dataset_dict)
len(loaded_dataset)

5561

In [11]:
from scripts.collect_binding_id_states import ExperimentResults

cache_path = os.path.join(
    env_utils.DEFAULT_RESULTS_DIR, 
    "binding_id_states",
    "Meta-Llama-3-70B-Instruct", 
    "results.json"
)
with open(cache_path, "r") as f:
    results = json.load(f)

In [12]:
res = ExperimentResults.from_dict(results)

In [17]:
res.cached_states[5].context_informed_actor['model.layers.79_<>_155'].shape

torch.Size([8192])